In [1]:
!pip install datasets
!pip install tqdm
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.3/760.3 kB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64

In [ ]:
import os
import shutil

# sample_data 폴더의 경로
sample_data_path = '/content/sample_data'

# sample_data 폴더가 존재하는지 확인
if os.path.exists(sample_data_path):
    # 폴더와 그 안의 모든 파일 및 디렉토리 삭제
    shutil.rmtree(sample_data_path)
    print('sample_data 폴더가 삭제되었습니다.')
else:
    print('sample_data 폴더가 존재하지 않습니다.')

sample_data 폴더가 삭제되었습니다.


In [ ]:
import os
print('현재 작업 경로 :', os.getcwd())

현재 작업 경로 : /content


In [ ]:
os.makedirs('./data/train/images')
os.makedirs('./data/train/labels')
os.makedirs('./data/validation/images')
os.makedirs('./data/validation/labels')

In [ ]:
def get_index(item, list):
    if item in list:
        return list.index(item)

# box[0], box[1], box[2], box[3] : xmin, ymin, xmax, ymax
# 좌표변환 (xyxy -> xywhn)
def convert_box(box):
          dw, dh = 1. / 1024, 1. / 576
          x, y, w, h = (box[0] + box[2]) / 2.0 - 1, (box[1] + box[3]) / 2.0 - 1, box[2] - box[0], box[3] - box[1]
          return[x * dw, y * dh, w * dw, h * dh]

# 소수 6자리로 변환
def format_to_6_decimal_places(number_list):
    output_list = []
    for number in number_list:
        formatted_number = round(number, 6)
        output_list.append(str(formatted_number))
    return output_list


In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
from datasets import load_dataset
ds = load_dataset("Hongik-ML-2024/processed-dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/31 [00:00<?, ?it/s]

In [ ]:
import json

# 미리 생성해둔 object_list.json 작업 경로 내 로드 (After make_object_list.py run)
with open('object_list.json', "r") as f:
    object_list = json.load(f)

In [ ]:
# train data 전처리
from tqdm import tqdm

train = ds['train']
train_data_dir = './data/train'
index = 0

for train_data in tqdm(train):
    # image 저장
    binary_data = train_data['image']
    with open(train_data_dir + f'/images/{index}.jpg', 'wb') as file:
        file.write(binary_data)

    # label text 생성 후 저장
    label_text = ''
    label = train_data['label']
    Annotations = label['Learning_Data_Info.']['Annotations']
    for annotation in Annotations:
        # 사물 index 알아내기
        object_num = get_index(annotation['Class_ID'], object_list)
        object_num = str(object_num)

        # 좌표 변환 (xyxy -> xywhn)
        box = annotation['Type_value']
        box = convert_box(box)
        box = format_to_6_decimal_places(box)
        coordinate = ' '.join(box)

        # 한줄 추가
        text = object_num + ' ' + coordinate
        label_text += text + '\n'
    label_text = label_text[:-1]

    with open(train_data_dir + f'/labels/{index}.txt', "w") as file:
        file.write(label_text)
    index += 1

100%|██████████| 586632/586632 [06:47<00:00, 1440.27it/s]


In [ ]:
# validation data 전처리
val = ds['validation']
val_data_dir = './data/validation'

index = 0
for val_data in tqdm(val):
    # image 저장
    binary_data = val_data['image']
    with open(val_data_dir + f'/images/{index}.jpg', 'wb') as file:
        file.write(binary_data)

    # label text 생성 후 저장
    label_text = ''
    label = val_data['label']
    Annotations = label['Learning_Data_Info.']['Annotations']
    for annotation in Annotations:
        # 사물 index 알아내기
        object_num = get_index(annotation['Class_ID'], object_list)
        object_num = str(object_num)

        # 좌표 변환 (xyxy -> xywhn)
        box = annotation['Type_value']
        box = convert_box(box)
        box = format_to_6_decimal_places(box)
        coordinate = ' '.join(box)

        # 한줄 추가
        text = object_num + ' ' + coordinate
        label_text += text + '\n'
    label_text = label_text[:-1]

    with open(val_data_dir + f'/labels/{index}.txt', "w") as file:
        file.write(label_text)
    index += 1

100%|██████████| 73379/73379 [01:09<00:00, 1060.90it/s]


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov9c.pt")

results = model.train(
   data='data/data.yaml',
   imgsz=1024,
   epochs=1,
   batch=8,
   name='yolov9c_custom'
)